Para usar este script es necesario configurar las siguientes environment variables:
- `AWS_ACCESS_KEY`
- `AWS_SECRET_ACCESS_KEY`
- `AWS_DEFAULT_REGION`

Cargando dependencias y conectando al recurso dynamoDB

In [1]:
import boto3
import pandas as pd

dynamodb= boto3.resource('dynamodb')

# Creando Tabla 'Ordenes'

In [5]:
ordenes = dynamodb.create_table(
    TableName = 'Ordenes',
    KeySchema =[
        {
            'AttributeName' : 'OrdenID',
            'KeyType' : 'HASH'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName' : 'OrdenID',
            'AttributeType' : 'S'
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits' : 10,
        'WriteCapacityUnits' : 10
    }
)

ordenes.wait_until_exists()

print(ordenes.item_count)

0


Agregando un Item

In [ ]:
ordenes.put_item(
    Item = {
        'OrdenID' : "189440",
        'Cliente' : [
            509477760, "Hertha Jiles",  "hjiles@ed.gov",     "Hombre",      "New Jersey",           False,                  30
        #   ClienteID   #Nombre          #Email                #Genero            #Lugar      #Internacional o no     #Edad
        ],
        'Producto' : [
            "BF8238",        1,              4
        #   ProductID       #Cantidad       #Rating
        ],
        "Precio" : [
            100,                0,              100
        #   PrecioSales        #Shipping        #Total
        ],
        'Reseña' :'The delivery team handled the product with care.'
    }
)

{'ResponseMetadata': {'RequestId': 'NI7FGLEGALJ20062T9IJFF1GKNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 25 Jan 2025 11:14:08 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'NI7FGLEGALJ20062T9IJFF1GKNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [8]:
response = ordenes.get_item(
    Key = {
        'OrdenID' : "189440"
    }
)

item = response['Item']
print(item)

{'OrdenID': '189440', 'Producto': ['BF8238', Decimal('1'), Decimal('4')], 'Cliente': [Decimal('509477760'), 'Hertha Jiles', 'hjiles@ed.gov', 'Hombre', False, Decimal('30')], 'Reseña': 'The delivery team handled the product with care.'}


# Creando tabla 'Productos'

In [10]:
productos = dynamodb.create_table(
    TableName = 'Productos',
    KeySchema =[
        {
            'AttributeName' : 'ProductID',
            'KeyType' : 'HASH'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName' : 'ProductID',
            'AttributeType' : 'S'
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits' : 10,
        'WriteCapacityUnits' : 10
    }
)

productos.wait_until_exists()

print(productos.item_count)

0


In [13]:
with productos.batch_writer() as batch:
    batch.put_item(
        Item = {
            'ProductID' : "BF9708",
            'Producto' : "Bufanda1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Bufanda uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/bufanda1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "Bf1414",
            'Producto' : "Camisa1",
            'Categoria' : "Prenda",
            'Descripcion' :'Camisa uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/camisa1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF6604",
            'Producto' : "Collar1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Collar uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/collar1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF2887",
            'Producto' : "Zapatos1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Zapato uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/zapatos1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF8063",
            'Producto' : "Vestido1",
            'Categoria' : "Prenda",
            'Descripcion' :'Vestido uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/vestido1.JPG"
        }
    )

In [26]:
from boto3.dynamodb.conditions import Key, Attr
response = productos.scan(
    FilterExpression = Attr('Categoria').eq('Prenda')
)

items = response['Items']
print(items)

[{'ProductID': 'BF8063', 'Categoria': 'Prenda', 'Producto': 'Vestido1', 'Descripcion': 'Vestido uno', 'imgURL': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/vestido1.JPG'}, {'ProductID': 'Bf1414', 'Categoria': 'Prenda', 'Producto': 'Camisa1', 'Descripcion': 'Camisa uno', 'imgURL': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/camisa1.JPG'}]
